# WMATA Trip Analysis in Jupyter Notebook

In [1]:
import pandas as pd
import numpy as np
import datetime

# Read in CSV data
data = pd.read_csv('https://raw.githubusercontent.com/conwayyao/WMATA-Trip-Analysis/master/trip_data.csv')

In [2]:
# Convert time strings to timestamps
time_converted = pd.to_datetime(data.Datetime, format="%m/%d/%Y %H:%M")
data.Datetime = time_converted

In [3]:
# Group into trips based on "Entry"
data['Trip'] = 'NaN'
data.sort_values(by = 'Datetime', ascending=True, inplace=True)
i=0
trips= []
for row in data.Description:
    if row == "Entry":
        i += 1    
    trips.append(i)
data['Trip'] = trips

In [9]:
# Calculate subway-to-subway trips
subway_trips=[]
for i in range(1, len(set(data['Trip']))+1):
    this_df = data[(data['Trip'] == i) & (data['Operator'] == 'Metrorail') & ( (data['Description'] == 'Entry') | (data['Description'] == 'Transfer') | (data['Description'] == 'Exit'))]
    if len(this_df) < 2:
        continue
    this_trip = {'trip': i, 'entr_trfr':this_df.iloc[0]['Description']}
    this_trip['entr_trfr'] = this_df.iloc[0]['Description']
    if this_df.iloc[0]['Description'] == 'Transfer':
        this_trip['origin'] = this_df.iloc[0]['Exit']
    else:
        this_trip['origin'] = this_df.iloc[0]['Entry_Route']
    this_trip['destination'] = this_df.iloc[-1]['Exit']
    this_trip['entry_time'] = this_df.iloc[0]['Datetime']
    this_trip['exit_time'] = this_df.iloc[-1]['Datetime']
    this_trip['weekday'] = this_df.iloc[0]['Weekday']
    this_trip['duration'] = this_df.iloc[-1]['Datetime'] - this_df.iloc[0]['Datetime']
    this_trip['cost'] = this_df.iloc[0]['Balance'] - this_df.iloc[1]['Balance']
    this_trip['month'] = this_df.iloc[0]['Month']
    subway_trips.append(this_trip)
    
columns = ['trip', 'month', 'weekday', 'entr_trfr', 'origin', 'destination', 'entry_time', 'exit_time', 'duration', 'cost']
subway_df = pd.DataFrame(subway_trips, columns=columns)
subway_df.set_index('trip', inplace=True)

In [10]:
# Look at longest-duration trips
subway_df.sort_values(by='duration').tail(5)

,month,weekday,entr_trfr,origin,destination,entry_time,exit_time,duration,cost
trip,,,,,,,,,
144,2,Sunday,Entry,New York Ave,Pentagon,2015-02-01 23:02:00,2015-02-02 00:17:00,0 days 01:15:00,2.10
133,1,Sunday,Transfer,Pentagon,L'Enfant Plaza,2015-01-25 11:41:00,2015-01-25 13:26:00,0 days 01:45:00,1.25
184,5,Friday,Transfer,Pentagon,Pentagon,2015-05-08 11:32:00,2015-05-08 13:21:00,0 days 01:49:00,1.25
188,5,Thursday,Transfer,Pentagon,Pentagon,2015-05-21 17:17:00,2015-05-21 21:35:00,0 days 04:18:00,1.65
269,11,Friday,Transfer,Pentagon,McPherson Square,2015-11-13 16:24:00,2015-11-17 17:54:00,4 days 01:30:00,1.70


In [11]:
# Drop erroneous data rows
subway_df.drop([184, 188, 269], inplace=True)

In [12]:
# Period of day
pd_list = []
for i in subway_df.entry_time:
    if (17 < i.hour < 22):
        pd_list.append('evening')
    elif (6 < i.hour < 11):
        pd_list.append('morning')
    elif (11 < i.hour < 17):
        pd_list.append('midday')
    else:
        pd_list.append('night')
subway_df['period'] = pd_list

In [13]:
# Examine subway trips
subway_df.columns
sum(subway_df.cost) # Total cost
len(subway_df) # Number of trips
len(subway_df.origin.value_counts())
subway_df.period.value_counts()
subway_df.duration.describe() # Total time
subway_df.sort_values(by='duration')
subway_df.sort_values(by=['origin', 'destination'])[['origin', 'destination', 'duration']]


,origin,destination,duration
trip,,,
165,Archives,Crystal City,00:31:00
213,Archives,Pentagon,00:16:00
68,Ballston,Rosslyn,00:12:00
64,Capitol South,Farragut West,00:16:00
9,Capitol South,Rosslyn,00:21:00
196,College Park,Pentagon,00:46:00
246,Columbia Heights,Pentagon,00:23:00
166,Crystal City,Pentagon,00:23:00
25,Dupont Circle,Gallery Place,00:08:00
